In [1]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [40]:
!pip install langchain

  Using cached frozenlist-1.5.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached multidict-6.1.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.0 kB)
  Using cached greenlet-3.1.1-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 306.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 297.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 727.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 802.4 kB/s eta 0:00:00a 0:00:01
Using cached frozenlist-1.5.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (283 kB)
Using cached greenlet-3.1.1-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (613 kB)
Using cached multidict-6.1.0-cp312-cp312-manylinux_2

In [11]:
pip install -qU langchain-anthropic

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install langchain_mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 859.9 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Imports

In [1]:
import os

from dotenv import load_dotenv
from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser

# Load API KEY

In [2]:
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

# Load model

In [3]:
model = ChatMistralAI(model="mistral-large-latest", api_key=MISTRAL_API_KEY)

In [12]:
 prompt_text = "Which Iranian football team has the most trophies?"
model.invoke(prompt_text)

AIMessage(content="As of my last update in October 2023, the Iranian football team with the most trophies is Persepolis F.C. Persepolis, based in Tehran, is one of the most successful clubs in Iranian football history. They have won numerous titles, including multiple Iran Pro League championships, Hazfi Cups, and other domestic and international competitions.\n\nFor the most current information, it's always a good idea to check the latest sports news or official club websites, as records and achievements can change over time.", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 15, 'total_tokens': 134, 'completion_tokens': 119}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-57e441f2-e1aa-4eb3-85c2-544dca3e4fc2-0', usage_metadata={'input_tokens': 15, 'output_tokens': 119, 'total_tokens': 134})

In [13]:
parser = StrOutputParser()

chain = model | parser
chain.invoke(prompt_text)

'As of my last update in October 2023, Persepolis FC is generally considered the Iranian football team with the most trophies. Persepolis has a rich history and has won numerous domestic and international titles, making it one of the most successful clubs in Iranian football.\n\nKey achievements of Persepolis FC include:\n- Multiple Iran Pro League titles\n- Hazfi Cup wins\n- Other domestic and international honors\n\nFor the most current and detailed information, it is advisable to check the latest football statistics and records.'

In [26]:
from langchain.prompts import ChatPromptTemplate

example_template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}  

Question: {question}
"""

example_prompt = ChatPromptTemplate.from_template(example_template)

In [27]:
chain = example_prompt | model | parser
chain.invoke({
    "context": "Kian is from Iran.",
    "question": "Where is Kian from?"
})

'Kian is from Iran.'

# Prompt Engineering

## In-context learning

### Zero Shot Learning

In [39]:
template_zero_shot = """
You are an expert sentiment classifier. Classify the following sentence as 
"Positive", "Negative", or "Neutral". 
Sentence: "{sentence}"
"""

prompt_zero_shot = ChatPromptTemplate.from_template(template_zero_shot)
chain_zero_shot = prompt_zero_shot | model | parser

response_zero_shot = chain_zero_shot.invoke({"sentence": "I love this movie."})
print(response_zero_shot)

The sentence "I love this movie." can be classified as **Positive**.


### One-shot Learning

In [48]:
template1 = """
You are an expert sentiment classifier. Classify the sentence below as "Positive", "Negative", or "Neutral",
based on the example provided.

Examples:
sentence: "This movie is awesome!"
Sentiment: Positive

Now, classify this new sentence:
"{sentence}"
"""

prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = prompt1 | model | parser
response = chain1.invoke({"sentence": "This movie was normal"})
print(response)


Based on the provided example and the sentence "This movie was normal," the sentiment can be classified as:

**Neutral**

The word "normal" does not convey a strong positive or negative emotion, indicating a neutral sentiment.


### Few-shot Learning

In [52]:
template1 = """
You are an expert sentiment classifier. Classify the sentence below as "Positive", "Negative", or "Neutral",
based on the examples provided.

Examples:
sentence: "This movie is awesome!"
Sentiment: Positive

sentence: "This is bad!"
Sentiment: Negative

sentence: "This movie was normal."
Sentiment: Neutral

Now, classify this new sentence:
"{sentence}"
"""

prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = prompt1 | model | parser
response = chain1.invoke({"sentence": "I love this movie."})
print(response)


Sentiment: Positive


### Role/Persona Prompts

In [58]:
template1 = """
Answer the question. If you can't answer the question, reply "I don't know".
Question: "{question}"
"""
prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = prompt1 | model | parser
response = chain1.invoke({"question": "?"})
print(response)

Babies are typically born through a process called childbirth. This process usually involves labor, where the mother experiences contractions that help push the baby out of the uterus and through the birth canal. The baby can be delivered vaginally or, in some cases, through a surgical procedure called a cesarean section (C-section). In a vaginal birth, the baby exits through the vagina, while in a C-section, the baby is delivered through an incision made in the mother's abdomen and uterus.


In [59]:
template1 = """
You are a kindergarden teacher for 5 year old students. One of the students asked you the question below, answer it so he/she can understand.

Question: "{question}"
"""

prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = prompt1 | model | parser
response = chain1.invoke({"question": "How do babies born?"})
print(response)

Sure! Babies grow inside a special place in their mommy's tummy called the womb. It's like a cozy little nest where the baby stays for about nine months. The baby gets bigger and stronger in the womb, and when it's ready, it comes out of mommy's tummy. Doctors and nurses help the mommy to make sure the baby comes out safely. It's a very special and amazing thing!


### Structured Output

In [66]:
template1 = """
You are an AI assistant that returns structured JSON. When asked a question, respond with a JSON object containing:
- "short_answer": a concise, one-sentence answer.
- "long_answer": a detailed explanation in at least 100 words.
- "sources": a list of any references cited.
question: "{question}"
"""

prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = prompt1 | model | parser
response = chain1.invoke({"question": "Where is Paris?"})
print(response)

```json
{
  "short_answer": "Paris is located in France.",
  "long_answer": "Paris, the capital city of France, is situated in the northern part of the country. It lies on the Seine River and is known for its rich history, iconic landmarks such as the Eiffel Tower and the Louvre Museum, and its significant cultural influence. Paris is also the most populous city in France and serves as a major European hub for finance, commerce, science, and the arts.",
  "sources": []
}
```
